In [1]:
import pandas as pd
import os

In [2]:
pwd = os.getcwd()


In [9]:
dataset = pd.read_excel(pwd + "/Data - Survey Monkey Output Edited.xlsx",sheet_name="Edited_Data")
dataset.head()

,Respondent ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,Identify which division you work in. - Response,Identify which division you work in. - Other (please specify),Which of the following best describes your position level? - Response,...,Question 29 - Response 8,Question 29 - Response 9,Question 29 - Response 10,Question 29 - Response 11,Question 29 - Response 12,Question 29 - Response 13,Question 29 - Response 14,Question 30 - Response 1,Question 30 - Response 2,Question 30 - Response 3
0,5379192392,2021-01-22 12:01:17,2021-01-22 12:40:34,NaN,NaN,NaN,NaN,Infrastructure,NaN,Staff,...,NaN,Answer 8,Answer 8,Answer 4,NaN,NaN,Answer 5,NaN,NaN,NaN
1,2658722536,2021-01-22 06:56:37,2021-01-22 07:34:10,NaN,NaN,NaN,NaN,Finance,NaN,Staff,...,NaN,Answer 5,NaN,NaN,Answer 2,NaN,Answer 5,NaN,NaN,Answer 1
2,4044163394,2021-01-22 06:35:18,2021-01-22 06:47:32,NaN,NaN,NaN,NaN,Infrastructure,NaN,Department Lead,...,NaN,NaN,Answer 4,Answer 4,Answer 6,NaN,Answer 6,NaN,Answer 1,NaN
3,5535865599,2021-01-21 21:29:32,2021-01-21 21:40:24,NaN,NaN,NaN,NaN,Infrastructure,NaN,Manager,...,Answer 2,Answer 5,Answer 7,NaN,Answer 6,NaN,Answer 7,Answer 7,Answer 1,Answer 6
4,3356802928,2021-01-21 17:26:39,2021-01-21 17:44:40,NaN,NaN,NaN,NaN,Port Operations,NaN,Manager,...,NaN,Answer 5,Answer 4,Answer 4,Answer 7,Answer 7,NaN,Answer 7,NaN,Answer 8


In [17]:
dataset_modified = dataset.copy()

In [18]:
cols_to_drop = ['Start Date', 'End Date', 'Email Address',
       'First Name', 'Last Name', 'Custom Data 1']

In [19]:
dataset_modified = dataset_modified.drop(columns=cols_to_drop)

In [27]:
id_vars = list(dataset_modified.columns[:8])
value_vars = list(dataset_modified.columns[8:])

In [30]:
dataset_melted = dataset_modified.melt(id_vars = id_vars, value_vars=value_vars, var_name="Question n Subquestion", value_name="Answer")

In [34]:
questions_import = pd.read_excel(pwd + "/Data - Survey Monkey Output Edited.xlsx",sheet_name="Question")
questions_import.head()

,Raw Question,Raw Subquestion,Question,Subquestion,Question + Subquestion
0,Respondent ID,NaN,Respondent ID,NaN,Respondent ID
1,Start Date,NaN,Start Date,NaN,Start Date
2,End Date,NaN,End Date,NaN,End Date
3,Email Address,NaN,Email Address,NaN,Email Address
4,First Name,NaN,First Name,NaN,First Name


In [39]:
questions = questions_import.copy()

In [40]:
questions=questions.drop(columns=['Raw Question', 'Raw Subquestion', 'Subquestion'])

In [49]:
questions = questions.rename(columns={"Question + Subquestion":"Question n Subquestion"})

In [50]:
# dataset_merged = pd.merge(left=left data set, right= right data set, hov= which type if join, left_on = cols from left to match, right_on = cols from right to match)

dataset_merged = pd.merge(left=dataset_melted, right=questions, how="left", left_on="Question n Subquestion",right_on="Question n Subquestion")
print("Original Data",len(dataset_melted))
print("Merged Data",len(dataset_merged))

Original Data 17028
Merged Data 17028


In [67]:
respondents = dataset_merged[dataset_merged["Answer"].notna()]
respondents = respondents.groupby('Question')['Respondent ID'].nunique().reset_index()
respondents.rename(columns={'Respondent ID':'Respondents'},inplace=True)

In [69]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondents, how="left", left_on="Question",right_on="Question")
print("Original Data",len(dataset_merged))
print("Merged Data",len(dataset_merged_two))

Original Data 17028
Merged Data 17028


In [78]:
same_answer = dataset_merged
same_answer = same_answer.groupby(['Question n Subquestion', 'Answer'])['Respondent ID'].nunique().reset_index()
same_answer.rename(columns={'Respondent ID':'Same Awnser'},inplace=True)
same_answer

,Question n Subquestion,Answer,Same Awnser
0,Question 1 - Response,Answer 1,14
1,Question 1 - Response,Answer 2,10
2,Question 1 - Response,Answer 3,13
3,Question 1 - Response,Answer 4,17
4,Question 1 - Response,Answer 5,22
...,...,...,...
683,Question 9 - Response 4,Answer 4,16
684,Question 9 - Response 4,Answer 5,13
685,Question 9 - Response 4,Answer 6,14
686,Question 9 - Response 4,Answer 7,12


In [81]:
dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left", left_on=["Question n Subquestion", 'Answer'],right_on=["Question n Subquestion", 'Answer'])
dataset_merged_three['Same Awnser'].fillna(0,inplace=True)
print("Original Data",len(dataset_merged_two))
print("Merged Data",len(dataset_merged_three))

Original Data 17028
Merged Data 17028


In [85]:
output = dataset_merged_three.copy()
output.rename(columns={"Identify which division you work in. - Response":"Division Primary","Identify which division you work in. - Other (please specify)":"Division Secondary","Which of the following best describes your position level? - Response":"Position","Which generation are you apart of? - Response":"Generation","Please select the gender in which you identify. - Response":"Gender","Which duration range best aligns with your tenure at your company? - Response":"Tenure","Which of the following best describes your employment type? - Response":"Employment Type"},inplace=True)
output

,Respondent ID,Division Primary,Division Secondary,Position,Generation,Gender,Tenure,Employment Type,Question n Subquestion,Answer,Question,Respondents,Same Awnser
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 - Response,Answer 4,Question 1,119,17.0
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5,Question 1,119,22.0
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1,Question 1,119,14.0
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8,Question 30,182,14.0
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6,Question 30,182,20.0
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0
